# 0. Setup WRDS API

###### To be done before running any related code
###### https://pypi.org/project/wrds/
###### Run the cell below
###### If it does not run, Mac: in terminal, Windows PC: in Ananconda (Jupyter, Spyder prompt) run "pip install wrds"

In [ ]:
!pip install wrds

# 1. Import WRDS Package

In [ ]:
import wrds

# 2. Connect with WRDS Server

In [ ]:
conn = wrds.Connection()

# 3. List of Libraries

###### "Library": Databases on WRDS, e.g. Compustat
###### list_libraries() function is to explore all subscribed databases
###### Check on what databases Utrecht University has subscription; if not subscribed it wouldn't work, e.g. CRSP

In [ ]:
conn.list_libraries().sort()
type(conn.list_libraries())

# 4. List of Datasets within a Given Library

###### Database contains sub datasets
###### list_tables() function lists all datasets within the database
###### Specifiy which "library/database"

In [ ]:
conn.list_tables(library='comp')

# 5. Query Data from WRDS Server

###### get_table() method
###### Straightforward if getting data from a single dataset
###### Specify which library/database, and table/dataset to "get"
###### Able to slice data by: Number of row, Column names

In [ ]:
# Extract first 5 obs from comp.company

company = conn.get_table(library='comp', table='company', obs=5)
company.shape

company

In [ ]:
# Narrow down the specific columns to extract

company_narrow = conn.get_table(library='comp', table='company', columns = ['conm', 'gvkey', 'cik'], obs=5)
company_narrow.shape

company_narrow

# 6. Subsetting Dataset

###### raw_sql() method; Similarto SQL syntax
###### When conditioning is needed
###### Can pre-specify date column format

In [ ]:
# Select one stock's monthly price
# from 2019 onwards

apple = conn.raw_sql("""select gvkey, datadate, fyear, at, sale, ni 
                        from comp.funda 
                        where gvkey = '16917'
                        and datadate>='2019-01-01'""", 
                     date_cols=['datadate'])

apple 

In [ ]:
apple.dtypes

# 7. Join Multiple Datasets

###### raw_sql() method: Syntax similar to proc sql in SAS
###### Handle conditioning statement

In [ ]:
apple_fund = conn.raw_sql("""select a.gvkey, a.iid, a.datadate, a.tic, a.conm,
                            a.at, b.prccm, b.cshoq 
                            
                            from comp.funda a 
                            inner join comp.secm b 
                            
                            on a.gvkey = b.gvkey
                            and a.iid = b.iid
                            and a.datadate = b.datadate
                        
                            where a.tic = 'AAPL' 
                            and a.datadate>='01/01/2010'
                            and a.datafmt = 'STD' 
                            and a.consol = 'C' 
                            and a.indfmt = 'INDL'
                            """, date_cols=['datadate'])

apple_fund.shape
apple_fund 

# 8. Saving Output

###### Flexible choice of output format
###### .csv or .xlsx / pickle for further python work / STATA format

In [ ]:
import pandas as pd

In [ ]:
# export the dataframe to csv format

apple_fund.to_csv('/your local directory/apple_fund.csv')

# export the dataframe to xlsx format

apple_fund.to_excel('/your local directory/apple_fund.xlsx')

In [ ]:
# pickle the dataframe

apple_fund.to_pickle("/your local directory/apple_fund.pkl")

In [ ]:
# export the dataframe to dta format for STATA

apple_fund.to_stata('/your local directory/apple_fund.dta')